Dropout
- this is another method of network regularization
- involves turning on and off some neurons in a layer in the network, called dropout layer, during training. This allows some inputs to the layer to pass through, while stopping others. Neurons to be turned off are randomly selected, with a given probability during each forward pass (i.e. 30% or 50% of neurons are turned off in a forward pass, neurons selected at random for each pass)
- forces the model to make use of all its neurons. This can help with:
    1) Prevents the network from becoming to reliant on any one neuron or reliance on a single neuron for a single instance.
    2) Preventing overfitting/memorizing the data as model cannot rely on specific neurons to memorize samples 
    3) Preventing co-adaptation - which is when a neuron becomes highly dependent on the outputs of another pre-synaptic neuron. Instead of learning the underlying function, it becomes specialized to the output of one of its input neurons. This can lead to poor generalization and overfitting
    4) Helpful if training data is noisy or has other perturbations. Dropout is its own form of noise - thus helping the network better generalize to noisy data. Exposing network to different subset of neurons during training teaches it to be more resilient to noise and variations in input data
- despite "turning off" some neurons, this does not speed up training process, as the dropped-out are not truly turned off, just their outputs are zeroed

Forward Pass
- firstly - bernoulli distribution and binomial distribtion. 
    1) A bernoulli distribution is the probability of a binary outcome for a single trial, like a coin toss, where outcomes are mutually exclusive. There is either heads or tails. In this scenario, heads =1, tails = 0. Probability of heads,p = .5; probability of tails = 1 - p. Generally p can anything between 0 and 1.
    2) a binomial distribtion is like the general case of a bernoulli distribtion. In general form, we can do multiple trials for outcomes from the distribution, summing them. So bernoulli distribution is a special case of binomial distribtion where a single trial is conducted. For example if we have a binomial distribtion with number of trials = 2, and p > 0. The possible range of values is 0, 1 , and 2 from this binomial distribtion. This is because when (p>0) and it is possible, in two trials, to draw a 1 each time. So summing these up, the binomial distribtion outcome is 0. It is 0 for the same reason, if p<1. and 1 may occur by drawing 1/0 or 0/1 on the trials. In the example given, (i.e. 1 or 0, where p = .5, n_samples = 2) there is 50% of a 1 (.5 + .5), and 25% of 0 or 2 (.5 * .5).
- to create our dropout, we will use a binomial distribution with n_samples = 1, and p = share of neurons we want to drop out. In some network frameworks, p may be defined oppositely, where it is the share of neurons being kept. It is just important to be aware of which way it is. We can use numpy to create an array with individual trials in each value with our desired binomial setting. See the example 1 code below - the function creates an array which should show at least one 0 out of the five positions in the array, but this will not always be the case as it is probabilistic. So on average, there will be four 1s and one 0. See example 2, it is the basic method by which dropout will be implemented.
- just dropping half the neurons presents an issue because we will be cutting the output by approximately the share of neurons that we drop. So similar to the normalization for adam, we need to divide by the (1 - dropout_rate) to rescale the dropout outputs back to their full values. So this way, we don't have to account for the dropout during prediction, and with enough samples, this scaling should average out to the approximate amount it would be without dropout. See example 3, notice how after scaling across many samples, the dropout sum is very close to the actual sum without dropout (intial sum)


In [6]:
import numpy as np

##Example 1
dropout_rate = 0.20 #how many should be 0
print("Example 1 output: ",np.random.binomial(1, 1-dropout_rate, size=5))

##Example 2
import numpy as np
dropout_rate = 0.3
example_output = np.array([0.27, -1.03, 0.67, 0.99, 0.05, -0.37, -2.01, 1.13, -0.07, 0.73])
example_output *= np.random.binomial(1, 1-dropout_rate, example_output.shape) #final argument is called size in the function, but we need to pass shape. It's confusing.
print("Example 2 output: ",example_output)

##Example 3
print("Example 3 output")
import numpy as np
dropout_rate = 0.2
example_output = np.array([0.27, -1.03, 0.67, 0.99, 0.05, -0.37, -2.01, 1.13, -0.07, 0.73])
print(f'sum initial {sum(example_output)}')
sums = []
for i in range(10000):
    example_output2 = example_output * np.random.binomial(1, 1-dropout_rate, example_output.shape) / (1-dropout_rate)
    sums.append(sum(example_output2))
print(f'mean sum: {np.mean(sums)}')

Example 1 output:  [1 1 1 1 1]
Example 2 output:  [ 0.   -0.    0.    0.99  0.05 -0.   -0.    1.13 -0.07  0.73]
Example 3 output
sum initial 0.36000000000000015
mean sum: 0.3605887500000002


Backward Pass
- We have done two changes to the dropout layer that must be accounted for
    1) as described, during dropout, a neuron can have two states, dropped or not dropped, so the neuron's output is either the output or 0
    2) then, we have also normalized this output so that it is closer to the true sum (see discussion above).

- we'll start with the scalar derivative (dividing by (1- dropout)). At first glance it may seem like quotient rule, but actually we just have a linear equation: f(x) = 1/(1 - q)*x => f'(x) = 1/(1-q), where 1 = dropout rate. We just treat the 1/(1-q) as a constant, so the derivative becomes simple dropping the linear variable. Note, that we use this derivative for neurons that are not dropped in the dropout layer.
- for neurons that are dropped, the derivative is 0. Intuitively, this makes sense, they are not contributing at all to loss.
- Please note that the dropout itself does not have any neurons, it just goes after a given layer in the network to turn off some neurons. So that's why the derivative is 1/(1-q) for not dropped neurons => their weight/input/bias derivatives are handled in layer before the dropout layer (from a forward pass perspective).
- full dropout derivative: where droput is D(z) and ri is the 1/0 mask for dropping neurons, z is the input from previous layer, q is dropout rate; 
    - D(z) = z/(1-q) where ri=1, 0 where ri=0
    - D'(z) = 1/(1-q) where ri=1, 0 where ri=0

New Dropout Layer
- the binary mask variable is the same as the derivative of the layer, so instead of writing code for the derivative, we take the binary mask used in the forward pass and just re-use in on the backward pass. That is why it is just dvalues (the values passed in from previous layer multiplied by binary mask)

In [ ]:
# Dropout
class Layer_Dropout:
    # Init
    def __init__(self, rate):
        # Store rate, we invert it as for example for dropout
        # of 0.1 we need success rate of 0.9
        self.rate = 1 - rate
        # Forward pass
    def forward(self, inputs):
        # Save input values
        self.inputs = inputs
        # Generate and save scaled mask
        self.binary_mask = np.random.binomial(1, self.rate,
        size=inputs.shape) / self.rate
        # Apply mask to output values
        self.output = inputs * self.binary_mask
        # Backward pass
    def backward(self, dvalues):
        # Gradient on values
        self.dinputs = dvalues * self.binary_mask

Dropout performance:
- run the final cell in the workbook for this cell to work
- with 64 neuron sized layers, the network has 68.8% testing accuracy and 75.7% validation accuracy. So validation performs better becuase the network is able to use all its neurons. Typically validation accuracy is higher than training with dropout for this reason.
- greater network size (not shown), with 512 per layer, still performs worse on validation than no dropout version, and training accuracy is slightly higher than validation, so one might suspect overfitting.

In [3]:
# Create dataset
X, y = spiral_data(samples=1000, classes=3)
# Create Dense layer with 2 input features and 64 output values
dense1 = Layer_Dense(2, 64, weight_regularizer_l2=5e-4,
bias_regularizer_l2=5e-4)
# Create ReLU activation (to be used with Dense layer):
activation1 = Activation_ReLU()
# Create dropout layer
dropout1 = Layer_Dropout(0.1)
# Create second Dense layer with 64 input features (as we take output
# of previous layer here) and 3 output values (output values)
dense2 = Layer_Dense(64, 3)
# Create Softmax classifier's combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
# Create optimizer
optimizer = Optimizer_Adam(learning_rate=0.05, decay=5e-5)
# Train in loop
for epoch in range(10001):
# Perform a forward pass of our training data through this layer
    dense1.forward(X)
    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1.output)
    # Perform a forward pass through Dropout layer
    dropout1.forward(activation1.output)
    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs
    dense2.forward(dropout1.output)
    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    data_loss = loss_activation.forward(dense2.output, y)
    # Calculate regularization penalty
    regularization_loss = loss_activation.loss.regularization_loss(dense1) + loss_activation.loss.regularization_loss(dense2)
    # Calculate overall loss
    loss = data_loss + regularization_loss
    # Calculate accuracy from output of activation2 and targets
    # calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1)
    
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions == y)
    
    if not epoch % 100:
        print(f'epoch: {epoch}, ' + f'acc: {accuracy:.3f}, ' + f'loss: {loss:.3f} (' + f'data_loss: {data_loss:.3f}, ' + f'reg_loss: {regularization_loss:.3f}), ' + f'lr: {optimizer.current_learning_rate}')
    
    # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    dropout1.backward(dense2.dinputs)
    activation1.backward(dropout1.dinputs)
    dense1.backward(activation1.dinputs)
    # Update weights and biases
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()

# Validate the model
# Create test dataset
X_test, y_test = spiral_data(samples=100, classes=3)
# Perform a forward pass of our testing data through this layer
dense1.forward(X_test)
# Perform a forward pass through activation function
# takes the output of first dense layer here
activation1.forward(dense1.output)
# Perform a forward pass through second Dense layer
# takes outputs of activation function of first layer as inputs
dense2.forward(activation1.output)
# Perform a forward pass through the activation/loss function
# takes the output of second dense layer here and returns loss
loss = loss_activation.forward(dense2.output, y_test)
# Calculate accuracy from output of activation2 and targets
# calculate values along first axis
predictions = np.argmax(loss_activation.output, axis=1)
if len(y_test.shape) == 2:
    y_test = np.argmax(y_test, axis=1)

accuracy = np.mean(predictions == y_test)
print(f'validation, acc: {accuracy:.3f}, loss: {loss:.3f}')

epoch: 0, acc: 0.324, loss: 1.099 (data_loss: 1.099, reg_loss: 0.000), lr: 0.05


epoch: 100, acc: 0.545, loss: 0.923 (data_loss: 0.900, reg_loss: 0.023), lr: 0.04975371909050202
epoch: 200, acc: 0.584, loss: 0.891 (data_loss: 0.863, reg_loss: 0.028), lr: 0.049507401356502806
epoch: 300, acc: 0.609, loss: 0.871 (data_loss: 0.844, reg_loss: 0.027), lr: 0.0492635105177595
epoch: 400, acc: 0.611, loss: 0.849 (data_loss: 0.823, reg_loss: 0.026), lr: 0.04902201088288642
epoch: 500, acc: 0.615, loss: 0.847 (data_loss: 0.822, reg_loss: 0.025), lr: 0.048782867456949125
epoch: 600, acc: 0.632, loss: 0.839 (data_loss: 0.815, reg_loss: 0.024), lr: 0.04854604592455945
epoch: 700, acc: 0.604, loss: 0.830 (data_loss: 0.804, reg_loss: 0.026), lr: 0.048311512633460556
epoch: 800, acc: 0.652, loss: 0.823 (data_loss: 0.797, reg_loss: 0.026), lr: 0.04807923457858551
epoch: 900, acc: 0.631, loss: 0.804 (data_loss: 0.778, reg_loss: 0.025), lr: 0.04784917938657352
epoch: 1000, acc: 0.656, loss: 0.801 (data_loss: 0.776, reg_loss: 0.026), lr: 0.04762131530072861
epoch: 1100, acc: 0.628, lo

Full network code thru this chapter

In [2]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data
nnfs.init()
import matplotlib.pyplot as plt

# Dense layer
class Layer_Dense:
    # Layer initialization
    def __init__(self, n_inputs, n_neurons, weight_regularizer_l1=0, weight_regularizer_l2=0, 
                 bias_regularizer_l1=0, bias_regularizer_l2=0):
        
        # Initialize weights and biases
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        # Set regularization strength
        self.weight_regularizer_l1 = weight_regularizer_l1
        self.weight_regularizer_l2 = weight_regularizer_l2
        self.bias_regularizer_l1 = bias_regularizer_l1
        self.bias_regularizer_l2 = bias_regularizer_l2
    
    # Forward pass
    def forward(self, inputs):
        # Remember input values
        self.inputs = inputs
        # Calculate output values from input ones, weights and biases
        self.output = np.dot(inputs, self.weights) + self.biases
    # Backward pass
    def backward(self, dvalues):
        # Gradients on parameters
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        # Gradients on regularization
        # L1 on weights
        if self.weight_regularizer_l1 > 0:
            dL1 = np.ones_like(self.weights)
            dL1[self.weights < 0] = -1
            self.dweights += self.weight_regularizer_l1 * dL1
        # L2 on weights
        if self.weight_regularizer_l2 > 0:
            self.dweights += 2 * self.weight_regularizer_l2 * self.weights
            # L1 on biases
        if self.bias_regularizer_l1 > 0:
            dL1 = np.ones_like(self.biases)
            dL1[self.biases < 0] = -1
            self.dbiases += self.bias_regularizer_l1 * dL1
        # L2 on biases
        if self.bias_regularizer_l2 > 0:
            self.dbiases += 2 * self.bias_regularizer_l2 * self.biases
        # Gradient on values
        self.dinputs = np.dot(dvalues, self.weights.T)
# Dropout
class Layer_Dropout:
    # Init
    def __init__(self, rate):
        # Store rate, we invert it as for example for dropout
        # of 0.1 we need success rate of 0.9
        self.rate = 1 - rate
        # Forward pass
    def forward(self, inputs):
        # Save input values
        self.inputs = inputs
        # Generate and save scaled mask
        self.binary_mask = np.random.binomial(1, self.rate,
        size=inputs.shape) / self.rate
        # Apply mask to output values
        self.output = inputs * self.binary_mask
        # Backward pass
    def backward(self, dvalues):
        # Gradient on values
        self.dinputs = dvalues * self.binary_mask

# ReLU activation
class Activation_ReLU:
    # Forward pass
    def forward(self, inputs):
        self.inputs = inputs
        # Calculate output values from inputs
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        # Since we need to modify original variable,
        # let’s make a copy of values first
        self.dinputs = dvalues.copy()
        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

# Softmax activation
class Activation_Softmax:
# Forward pass
    def forward(self, inputs):
        # Remember input values
        self.inputs = inputs
        
        # Get unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        
        # Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        
        self.output = probabilities
    # Backward pass
    def backward(self, dvalues):
        # Create uninitialized array
        self.dinputs = np.empty_like(dvalues)
        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            # Flatten output array
            single_output = single_output.reshape(-1, 1)
            # Calculate Jacobian matrix of the output
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            # Calculate sample-wise gradient
            # and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

# Common loss class
class Loss:
# Regularization loss calculation
    def regularization_loss(self, layer):
    # 0 by default
        regularization_loss = 0
        # L1 regularization - weights
        # calculate only when factor greater than 0
        if layer.weight_regularizer_l1 > 0:
            regularization_loss += layer.weight_regularizer_l1 * np.sum(np.abs(layer.weights))
        # L2 regularization - weights
        if layer.weight_regularizer_l2 > 0:
            regularization_loss += layer.weight_regularizer_l2 * np.sum(layer.weights * layer.weights)
        # L1 regularization - biases
        # calculate only when factor greater than 0
        if layer.bias_regularizer_l1 > 0:
            regularization_loss += layer.bias_regularizer_l1 * np.sum(np.abs(layer.biases))
        # L2 regularization - biases
        if layer.bias_regularizer_l2 > 0:
            regularization_loss += layer.bias_regularizer_l2 * np.sum(layer.biases * layer.biases)
        return regularization_loss
    # Calculates the data and regularization losses
    # given model output and ground truth values
    def calculate(self, output, y):
        # Calculate sample losses
        sample_losses = self.forward(output, y)
        # Calculate mean loss
        data_loss = np.mean(sample_losses)
        # Return loss
        return data_loss
        
class Loss_CategoricalCrossentropy(Loss):
# Forward pass
    def forward(self, y_pred, y_true):
    # Number of samples in a batch
        samples = len(y_pred)
        
        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        # Probabilities for target values -
        # only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples),y_true]
        # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods
    
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)
        # Number of labels in every sample
        # We'll use the first sample to count them
        labels = len(dvalues[0])
        # If labels are sparse, turn them into one-hot vector
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
        
        # Calculate gradient
        self.dinputs = -y_true / dvalues
        # Normalize gradient
        self.dinputs = self.dinputs / samples


class Activation_Softmax_Loss_CategoricalCrossentropy():
# Creates activation and loss function objects
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()
        # Forward pass
    def forward(self, inputs, y_true):
        # Output layer's activation function
        self.activation.forward(inputs)
        # Set the output
        self.output = self.activation.output
        # Calculate and return loss value
        return self.loss.calculate(self.output, y_true)
    # Backward pass
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)

        # If labels are one-hot encoded,
        # turn them into discrete values
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)
        
        # Copy so we can safely modify
        self.dinputs = dvalues.copy()
    
        # For each row in dinputs, get what the network has for the correct class and subtract 1
        self.dinputs[range(samples), y_true] -= 1
        
        # Normalize gradient
        self.dinputs = self.dinputs / samples

class Optimizer_SGD:
# Initialize optimizer - set settings,
# learning rate of 1. is default for this optimizer
    def __init__(self, learning_rate=1., decay=0., momentum=0.):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum
    # Call once before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay*self.iterations))
    # Update parameters
    def update_params(self, layer):
    # If we use momentum
        if self.momentum:
        # If layer does not contain momentum arrays, create them
        # filled with zeros
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                # If there is no momentum array for weights
                # The array doesn't exist for biases yet either.
                layer.bias_momentums = np.zeros_like(layer.biases)
            
            # Build weight updates with momentum - take previous
            # updates multiplied by retain factor and update with
            # current gradients
            weight_updates = self.momentum * layer.weight_momentums - self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates
            # Build bias updates
            bias_updates = self.momentum * layer.bias_momentums - self.current_learning_rate * layer.dbiases
            layer.bias_momentums = bias_updates

        else:
            weight_updates = -self.current_learning_rate * layer.dweights
            bias_updates = -self.current_learning_rate * layer.dbiases

        layer.weights += weight_updates
        layer.biases += bias_updates

    # Call once after any parameter updates
    def post_update_params(self):
        self.iterations += 1

class Optimizer_Adagrad:
# Initialize optimizer - set settings,
# learning rate of 1. is default for this optimizer
    def __init__(self, learning_rate=1., decay=0., epsilon=1e-7):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
    # Call once before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay*self.iterations))
    # Update parameters
    def update_params(self, layer):
        # If layer does not contain cache arrays,
        # create them filled with zeros
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)
        # Update cache with squared current gradients
        layer.weight_cache += layer.dweights**2
        layer.bias_cache += layer.dbiases**2
        # Vanilla SGD parameter update + normalization
        # with square rooted cache
        layer.weights += -self.current_learning_rate * layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)
        layer.biases += -self.current_learning_rate * layer.dbiases / (np.sqrt(layer.bias_cache) + self.epsilon)

    # Call once after any parameter updates
    def post_update_params(self):
        self.iterations += 1

class Optimizer_RMSprop:
# Initialize optimizer - set settings,
# learning rate of 1. is default for this optimizer
    def __init__(self, learning_rate=.001, decay=0., epsilon=1e-7, rho=.9):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.rho = rho
    # Call once before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay*self.iterations))
    # Update parameters
    def update_params(self, layer):
        # If layer does not contain cache arrays,
        # create them filled with zeros
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)
        # Update cache with squared current gradients
        layer.weight_cache = self.rho * layer.weight_cache + (1 - self.rho) * layer.dweights**2
        layer.bias_cache = self.rho * layer.bias_cache + (1 - self.rho) * layer.dbiases**2
        # Vanilla SGD parameter update + normalization
        # with square rooted cache
        layer.weights += -self.current_learning_rate * layer.dweights / (np.sqrt(layer.weight_cache) + self.epsilon)
        layer.biases += -self.current_learning_rate * layer.dbiases / (np.sqrt(layer.bias_cache) + self.epsilon)

    # Call once after any parameter updates
    def post_update_params(self):
        self.iterations += 1

class Optimizer_Adam:
    # Initialize optimizer - set settings
    def __init__(self, learning_rate=0.001, decay=0., epsilon=1e-7, beta_1=0.9, beta_2=0.999):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.beta_1 = beta_1
        self.beta_2 = beta_2
    
    # Call once before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))
    
    # Update parameters
    def update_params(self, layer):
    # If layer does not contain cache arrays,
    # create them filled with zeros
        if not hasattr(layer, 'weight_cache'):
            layer.weight_momentums = np.zeros_like(layer.weights)
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_momentums = np.zeros_like(layer.biases)
            layer.bias_cache = np.zeros_like(layer.biases)
        
        # Update momentum with current gradients
        layer.weight_momentums = self.beta_1 * layer.weight_momentums + (1 - self.beta_1) * layer.dweights
        layer.bias_momentums = self.beta_1 * layer.bias_momentums + (1 - self.beta_1) * layer.dbiases
        
        # Get corrected momentum
        # self.iteration is 0 at first pass
        # and we need to start with 1 here
        weight_momentums_corrected = layer.weight_momentums / (1 - self.beta_1 ** (self.iterations + 1))
        bias_momentums_corrected = layer.bias_momentums / (1 - self.beta_1 ** (self.iterations + 1))
        
        # Update cache with squared current gradients
        layer.weight_cache = self.beta_2 * layer.weight_cache + (1 - self.beta_2) * layer.dweights**2

        layer.bias_cache = self.beta_2 * layer.bias_cache + (1 - self.beta_2) * layer.dbiases**2
        
        # Get corrected cache
        weight_cache_corrected = layer.weight_cache / (1 - self.beta_2 ** (self.iterations + 1))
        bias_cache_corrected = layer.bias_cache / (1 - self.beta_2 ** (self.iterations + 1))
        
        # Vanilla SGD parameter update + normalization
        # with square rooted cache
        layer.weights += -self.current_learning_rate * weight_momentums_corrected / (np.sqrt(weight_cache_corrected) + self.epsilon)
        layer.biases += -self.current_learning_rate * bias_momentums_corrected / (np.sqrt(bias_cache_corrected) + self.epsilon)
    
    # Call once after any parameter updates
    def post_update_params(self):
        self.iterations += 1